# Trivial compression
Saving space (virtual or real) is often important. It is more efficient to use less space, and it can save money. 
If you are renting an apartment that is bigger than you need for your things and family, you could “downsize” to a 
smaller place that is less expensive. If you are paying by the byte to store your data on a server, you may want to 
compress it so that its storage costs you less. Compression is the act of taking data and encoding it (changing its form) 
in such a way that it takes up less space. Decompression is reversing the process, returning the data to its original form.

If it is more storage-efficient to compress data, then why is all data not com- pressed? There is a trade-off between 
time and space. It takes time to compress a piece of data and to decompress it back into its original form. Therefore, 
data compression only makes sense in situations where small size is prioritized over fast execution. Think of large 
files being transmitted over the internet. Compressing them makes sense because it will take longer to transfer the 
files than it will to decompress them once received. Further, the time taken to compress the files for their storage on 
the original server only needs to be accounted for once.

The easiest data compression wins come about when you realize that data storage types use more bits than are strictly 
required for their contents. For instance, thinking low-level, if an unsigned integer that will never exceed 65,535 is 
being stored as a 64- bit unsigned integer in memory, it is being stored inefficiently. It could instead be stored as a 
16-bit unsigned integer. This would reduce the space consumption for the actual number by 75% (16 bits instead of 64 bits). 
If millions of such numbers are being stored inefficiently, it can add up to megabytes of wasted space.

In Python, sometimes for the sake of simplicity (which is a legitimate goal, of course), the developer is shielded from 
thinking in bits. There is no 64-bit unsigned integer type, and there is no 16-bit unsigned integer type. There is just 
a single int type that can store numbers of arbitrary precision. The function ```sys.getsizeof()``` can help you find out how 
many bytes of memory your Python objects are consuming. But due to the inherent overhead of the Python object system, 
there is no way to cre- ate an int that takes up less than 28 bytes (224 bits) in Python 3.7. A single int can be 
extended one bit at a time (as we will do in this example), but it consumes a minimum of 28 bytes.

>**NOTE** If you are a little rusty regarding binary, recall that a bit is a single value that is either a 1 or a 0. 
A sequence of 1s and 0s is read in base 2 to represent a number. For the purposes of this section, 
you do not need to do any math in base 2, but you do need to understand that the number of bits that a type stores 
determines how many different values it can represent. For example, 1 bit can represent 2 values (0 or 1), 2 bits can 
represent 4 values (00, 01, 10, 11), 3 bits can represent 8 values, and so on.

If the number of possible different values that a type is meant to represent is less than the number of values that the 
bits being used to store it can represent, it can likely be more efficiently stored. Consider the nucleotides that form 
a gene in DNA.2 Each nucleotide can only be one of four values: A, C, G, or T. Yet if the gene is stored as a str, 
which can be thought of as a collection of Unicode characters, each nucleotide will be represented by a character, 
which generally requires 8 bits of storage. 

In binary, just 2 bits are needed to store a type with four possible values: 00, 01, 10, and 11 are the four different 
values that can be represented by 2 bits. If A is assigned 00, C is assigned 01, G is assigned 10, and T is assigned 11, 
the storage required for a string of nucleotides can be reduced by 75% (from 8 bits to 2 bits per nucleotide).

Instead of storing our nucleotides as a str, they can be stored as a bit string. 
bit string is exactly what it sounds like: an arbitrary-length sequence of 1s and 0s. 
Unfortunately, the Python standard library contains no off-the-shelf construct for working with bit strings of 
arbitrary length. The following code converts a str composed of As, Cs, Gs, and Ts into a string of bits and back again. 
The string of bits is stored within an int. Because the int type in Python can be of any length, it can be used as a 
bit string of any length. To convert back into a str, we will implement the Python ```__str__()``` special method.

In [1]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)

A CompressedGene is provided a str of characters representing the nucleotides in a gene, and it internally stores the 
sequence of nucleotides as a bit string. The __init__() method’s main responsibility is to initialize the bit-string 
construct with the appropriate data. __init__() calls _compress() to do the dirty work of actually converting the 
provided str of nucleotides into a bit string.

Note that _compress() starts with an underscore. Python has no concept of truly private methods or variables. 
(All variables and methods can be accessed through reflection; there’s no strict enforcement of privacy.) A leading 
underscore is used as a convention to indicate that the implementation of a method should not be relied on by actors 
outside of the class. (It is subject to change and should be treated as private.)

>**TIP** If you start a method or instance variable name in a class with two lead- ing underscores, 
Python will “name mangle” it, changing its implementation name with a salt and not making it easily discoverable by 
other classes. We use one underscore in this book to indicate a “private” variable or method, but you may wish to use 
two if you really want to emphasize that something is pri- vate. For more on naming in Python, check out the section 
“Descriptive Nam- ing Styles” from PEP 8: http://mng.bz/NA52.

Next, let’s look at how we can actually perform the compression.

In [12]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)
        
    def _compress(self, gene: str) -> None: 
        self.bit_string: int = 1 # start with sentinel 
        for nucleotide in gene.upper():
            self.bit_string <<= 2 # shift left two bits
            if nucleotide == "A": # change last two bits to 00
                self.bit_string |= 0b00
            elif nucleotide == "C": # change last two bits to 01
                self.bit_string |= 0b01
            elif nucleotide == "G": # change last two bits to 10
                self.bit_string |= 0b10
            elif nucleotide == "T": # change last two bits to 11
                self.bit_string |= 0b11 
            else:
                raise ValueError("Invalid Nucleotide:{}".format(nucleotide))

The ```_compress()``` method looks at each character in the str of nucleotides sequentially. 
When it sees an A, it adds 00 to the bit string. When it sees a C, it adds 01, and so on. 
Remember that two bits are needed for each nucleotide. As a result, before we add each new nucleotide, 
we shift the bit string two bits to the left (self.bit_string <<= 2).

Every nucleotide is added using an “or” operation (|). After the left shift, two 0s are added to the right side of the 
bit string. In bitwise operations, “ORing” (for example, self.bit_string |= 0b10) 0s with any other value results in the 
other value replacing the 0s. In other words, we continually add two new bits to the right side of the bit string. 
The two bits that are added are determined by the type of the nucleotide.

Finally, we will implement decompression and the special ```__str__()``` method that uses it.

In [15]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)
        
    def _compress(self, gene: str) -> None: 
        self.bit_string: int = 1 # start with sentinel 
        for nucleotide in gene.upper():
            self.bit_string <<= 2 # shift left two bits
            if nucleotide == "A": # change last two bits to 00
                self.bit_string |= 0b00
            elif nucleotide == "C": # change last two bits to 01
                self.bit_string |= 0b01
            elif nucleotide == "G": # change last two bits to 10
                self.bit_string |= 0b10
            elif nucleotide == "T": # change last two bits to 11
                self.bit_string |= 0b11 
            else:
                raise ValueError("Invalid Nucleotide:{}".format(nucleotide))
            
    def decompress(self) -> str: 
        gene: str = ""
        for i in range(0, self.bit_string.bit_length() - 1, 2): # - 1 to exclude sentinel
            bits: int = self.bit_string >> i & 0b11 # get just 2 relevant bits 
            if bits == 0b00: # A
                gene += "A"
            elif bits == 0b01: # C
                gene += "C"
            elif bits == 0b10: # G
                gene += "G"
            elif bits == 0b11: # T
                gene += "T" 
            else:
                raise ValueError("Invalid bits:{}".format(bits))
        return gene[::-1] # [::-1] reverses string by slicing backward
    
    def __str__(self) -> str: # string representation for pretty printing 
        return self.decompress()

```decompress()``` reads two bits from the bit string at a time, and it uses those two bits to determine which character 
to add to the end of the str representation of the gene. Because the bits are being read backward, compared to the order 
they were com- pressed in (right to left instead of left to right), the str representation is ultimately reversed 
(using the slicing notation for reversal [::-1]). Finally, note how the convenient int method ```bit_length()``` aided in 
the development of ```decompress()```. Let’s test it out.

In [40]:
from sys import getsizeof 


original: str ="TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATA" * 100
original_size = getsizeof(original)
print("original is {} bytes".format(original_size))
compressed: CompressedGene = CompressedGene(original) # compress 
compressed_size = getsizeof(compressed.bit_string)
print("compressed is {} bytes".format(compressed_size)) 
print("compressed is {0:.0%} smaller than original".format(1 - compressed_size / original_size))
print(compressed) # decompress
print("original and decompressed are the same: {}".format(original == compressed.decompress()))

original is 8649 bytes
compressed is 2320 bytes
compressed is 73% smaller than original
TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTA

>**NOTE** In the CompressedGene class, we used if statements extensively to decide between a series of cases in both the 
compression and the decompression methods. Because Python has no switch statement, this is somewhat typical. 
What you will also see in Python sometimes is a high reliance on dictionaries in place of extensive if statements to 
deal with a set of cases. Imagine, for instance, a dictionary in which we could look up each nucleotide’s respective bits. 
This can sometimes be more readable, but it can come with a performance cost. Even though a dictionary lookup is technically O(1), 
the cost of running a hash function will sometimes mean a dictionary is less performant than a series of ifs. Whether 
this holds will depend on what a particular program’s if statements must evaluate to make their decision. 
You may want to run performance tests on both methods if you need to make a decision between ifs and dictionary lookup 
in a critical section of code.
